In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

In [12]:
%load_ext autoreload
%autoreload 2

import re
import urllib.request
api_url = 'https://raw.githubusercontent.com/tanmayyb/tmu-capstone-anomaly-detection/refs/heads/main/api/datasets.py'
exec(urllib.request.urlopen(api_url).read())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
ieso_dataset = load_ieso_dataset(2018, 2020, join=True).iloc[:-1]

In [14]:
weather_dataset = load_climate_dataset(2018, 2020, join=True).iloc[1:]

100%|██████████| 3/3 [01:11<00:00, 23.98s/it]


In [15]:
#@markdown merge and preprocess

dataset = pd.merge(ieso_dataset, weather_dataset, on="DateTime")

def preprocess(dataset:pd.DataFrame, split_datetime=True) -> pd.DataFrame:
  df = dataset.copy()
  ieso_cols = ['Toronto']
  climate_cols = [
       'Temp (°C)',
      #  'Temp Flag',
       'Dew Point Temp (°C)',
      #  'Dew Point Temp Flag',
       'Rel Hum (%)',
      #  'Rel Hum Flag',
       'Precip. Amount (mm)',
      #  'Precip. Amount Flag',
      #  'Wind Dir (10s deg)',
      #  'Wind Dir Flag',
      #  'Wind Spd (km/h)',
      #  'Wind Spd Flag',
      #  'Visibility (km)',
      #  'Visibility Flag',
       'Stn Press (kPa)',
      #  'Stn Press Flag',
       'Hmdx',
      #  'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather'
      ]
  if split_datetime:
    df['Y'] = df['DateTime'].dt.year
    df['M'] = df['DateTime'].dt.month
    df['D'] = df['DateTime'].dt.day
    df['H'] = df['DateTime'].dt.hour
    cols = ['Y', 'M', 'D', 'H']
  else:
    cols = ['DateTime']

  # delete leap day
  df = df[~((df.DateTime.dt.month == 2) & (df.DateTime.dt.day == 29))]
  dt = df['DateTime'] # store datettime

  cols += ieso_cols+climate_cols

  df = df[cols]

  # make columns names better
  df.columns = df.columns.str.replace('.', '')
  df.columns = df.columns.str.replace(' ', '')
  df.columns = df.columns.str.replace(r"\(.*?\)", "", regex=True)

  nans = df.isna().sum().to_dict()

  # dirty mean imputation
  data = df.fillna(df.mean())

  # # dirty -1 imputation
  # data = df.fillna(pd.Series(index=df.columns, data=[-1.0]*len(df.columns)))

  data = data.reset_index()

  return df, nans, dt

df, nans, dt = preprocess(dataset)

In [56]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense


data = df[['Toronto']]


scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

train_size = int(len(data_scaled) * 0.80)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]


def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 3
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')


model.fit(X_train, Y_train, epochs=50, batch_size=200, verbose=0)


train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


train_predict = scaler.inverse_transform(np.concatenate((train_predict, np.zeros((len(train_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_train = scaler.inverse_transform(np.concatenate((Y_train.reshape(-1, 1), np.zeros((len(Y_train), len(data.columns) - 1))), axis=1))[:, 0]
test_predict = scaler.inverse_transform(np.concatenate((test_predict, np.zeros((len(test_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_test = scaler.inverse_transform(np.concatenate((Y_test.reshape(-1, 1), np.zeros((len(Y_test), len(data.columns) - 1))), axis=1))[:, 0]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



657/657 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [ ]:
# prompt: write the same code but without any look_back parameters

import numpy as np
def create_dataset(dataset):
    X, Y = [], []
    for i in range(len(dataset) - 1):
        a = dataset[i, 0]
        X.append(a)
        Y.append(dataset[i + 1, 0])
    return np.array(X), np.array(Y)

X_train, Y_train = create_dataset(train_data)
X_test, Y_test = create_dataset(test_data)

X_train = np.reshape(X_train, (X_train.shape[0], 1, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 1, 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, Y_train, epochs=50, batch_size=200, verbose=0)

train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

train_predict = scaler.inverse_transform(np.concatenate((train_predict, np.zeros((len(train_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_train = scaler.inverse_transform(np.concatenate((Y_train.reshape(-1, 1), np.zeros((len(Y_train), len(data.columns) - 1))), axis=1))[:, 0]
test_predict = scaler.inverse_transform(np.concatenate((test_predict, np.zeros((len(test_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_test = scaler.inverse_transform(np.concatenate((Y_test.reshape(-1, 1), np.zeros((len(Y_test), len(data.columns) - 1))), axis=1))[:, 0]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [ ]:
print(len(Y_test))

In [62]:
#@markdown plot prediction
fig = go.Figure()
fig.add_trace(go.Scattergl(
    x=dt[train_size + look_back + 1:][:len(Y_test)],
    y=Y_test,
    name='Actual',
    line_color='blue')
)

fig.add_trace(go.Scattergl(
    x=dt[train_size + look_back + 1:][:len(Y_test)],
    y=test_predict,
    name='Predicted',
    line_color='red')
)

# Set the theme to 'plotly_white'
fig.update_layout(
    title=f"Time Series Forecasting for with LSTM",
    xaxis_title="Date", # Change the x-axis title
    yaxis_title="Energy Demand",
    template="plotly_white",
    xaxis = dict(
      rangeslider=dict(
          visible=True
      ),
      tickformat="%Y-%m-%d"
    )
)
fig.show()


In [63]:
tmp = pd.concat([dt, pd.Series(Y_test, name='Y_test'), pd.Series(test_predict, name='pred')], axis=1).dropna()

In [64]:
tmp

,DateTime,Y_test,pred
0,2018-01-01 01:00:00,4981.0,4937.117944
1,2018-01-01 02:00:00,4866.0,4891.305535
2,2018-01-01 03:00:00,4925.0,4822.102992
3,2018-01-01 04:00:00,5186.0,5057.516496
4,2018-01-01 05:00:00,5813.0,5480.821479
...,...,...,...
26274,2020-12-30 19:00:00,0.0,0.000000
26275,2020-12-30 20:00:00,0.0,0.000000
26276,2020-12-30 21:00:00,0.0,0.000000
26277,2020-12-30 22:00:00,0.0,0.000000


In [50]:
Y_test

array([4981., 4866., 4925., ..., 5948., 5741., 5527.])

In [52]:
# prompt: how many rows is Y_test have

print(len(Y_test))
print(len(test_predict))
print(len(df))


5252
5252
26279


In [18]:
# prompt: make the filtered_dt start the row with 0

# Filter the data based on the specified date range
start_date = '2018-10-20 00:00:00'
end_date = '2020-12-31 23:00:00'
mask = (dt >= start_date) & (dt <= end_date)

# Apply the mask to the relevant variables
filtered_dt = dt[mask].reset_index(drop=True) # Reset index here
filtered_Y_test = Y_test[(dt[train_size + look_back + 1:] >= start_date) & (dt[train_size + look_back + 1:] <= end_date)]
filtered_test_predict = test_predict[(dt[train_size + look_back + 1:] >= start_date) & (dt[train_size + look_back + 1:] <= end_date)]


# Create the filtered plot
fig = go.Figure()
fig.add_trace(go.Scattergl(
    x=filtered_dt[:len(filtered_Y_test)],
    y=filtered_Y_test,
    name='Actual',
    line_color='blue')
)

fig.add_trace(go.Scattergl(
    x=filtered_dt[:len(filtered_Y_test)],
    y=filtered_test_predict,
    name='Predicted',
    line_color='red')
)

# Set the theme to 'plotly_white'
fig.update_layout(
    title=f"Time Series Forecasting for with LSTM (2018-10-20 to 2020-12-31)",
    xaxis_title="Date", # Change the x-axis title
    yaxis_title="Energy Demand",
    template="plotly_white",
    xaxis = dict(
      rangeslider=dict(
          visible=True
      ),
      tickformat="%Y-%m-%d"
    )
)
fig.show()


In [20]:
filtered_dt

,DateTime
0,2018-10-20 00:00:00
1,2018-10-20 01:00:00
2,2018-10-20 02:00:00
3,2018-10-20 03:00:00
4,2018-10-20 04:00:00
...,...
19267,2020-12-31 19:00:00
19268,2020-12-31 20:00:00
19269,2020-12-31 21:00:00
19270,2020-12-31 22:00:00


In [9]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error


mse = mean_squared_error(Y_test, test_predict)
print(f'Mean Squared Error (MSE): {mse}')


mae = mean_absolute_error(Y_test, test_predict)
print(f'Mean Absolute Error (MAE): {mae}')


mape = np.mean(np.abs((Y_test - test_predict) / Y_test)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')


Mean Squared Error (MSE): 10848.803490231983
Mean Absolute Error (MAE): 76.50171507861478
Mean Absolute Percentage Error (MAPE): 1.34%


In [12]:
# ipython-input-195-3ed81542a0a7
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [78]:
import pandas as pd
tmp = pd.concat([filtered_dt, pd.Series(filtered_Y_test, name='Y_test'), pd.Series(filtered_test_predict, name='pred')], axis=1).dropna()

tmp['error'] = tmp['Y_test'] - tmp['pred']
tmp['abs_error'] = tmp['error'].abs()

worst_predicted = tmp.sort_values(by='abs_error', ascending=False)
worst_predicted[:10]

,DateTime,Y_test,pred,error,abs_error
1021,2018-12-01 13:00:00,9018.0,8148.909330,869.090670,869.090670
1483,2018-12-20 19:00:00,7752.0,8302.544473,-550.544473,550.544473
1475,2018-12-20 11:00:00,9266.0,8792.944640,473.055360,473.055360
3251,2019-03-04 11:00:00,5396.0,4940.523768,455.476232,455.476232
4120,2019-04-09 16:00:00,5762.0,6198.433289,-436.433289,436.433289
1282,2018-12-12 10:00:00,7526.0,7953.583280,-427.583280,427.583280
4480,2019-04-24 16:00:00,5929.0,6354.838569,-425.838569,425.838569
4456,2019-04-23 16:00:00,5941.0,6355.993313,-414.993313,414.993313
1012,2018-12-01 04:00:00,6974.0,6560.580642,413.419358,413.419358
4024,2019-04-05 16:00:00,6100.0,6504.841971,-404.841971,404.841971


In [21]:
import pandas as pd
tmp = pd.concat([dt, pd.Series(Y_test, name='Y_test'), pd.Series(test_predict, name='pred')], axis=1).dropna()

tmp['error'] = tmp['Y_test'] - tmp['pred']
tmp['abs_error'] = tmp['error'].abs()

In [42]:
tmp

,DateTime,Y_test,pred,error,abs_error
0,2018-10-20 00:00:00,4981.0,4943.808553,37.191447,37.191447
1,2018-10-20 01:00:00,4866.0,4901.202042,-35.202042,35.202042
2,2018-10-20 02:00:00,4925.0,4833.083989,91.916011,91.916011
3,2018-10-20 03:00:00,5186.0,5070.713492,115.286508,115.286508
4,2018-10-20 04:00:00,5813.0,5492.356062,320.643938,320.643938
...,...,...,...,...,...
5247,2019-05-26 15:00:00,6198.0,6054.359545,143.640455,143.640455
5248,2019-05-26 16:00:00,6123.0,6476.840117,-353.840117,353.840117
5249,2019-05-26 17:00:00,5948.0,5941.870355,6.129645,6.129645
5250,2019-05-26 18:00:00,5741.0,5767.098088,-26.098088,26.098088


In [40]:
tmp.to_csv('/content/drive/My Drive/' + 'lstm_Toronto_2.csv', index=False)

In [ ]:
#@markdown merge and preprocess

dataset = pd.merge(ieso_dataset, weather_dataset, on="DateTime")

def preprocess(dataset:pd.DataFrame, split_datetime=True) -> pd.DataFrame:
  df = dataset.copy()
  ieso_cols = ['Ottawa']
  climate_cols = [
       'Temp (°C)',
      #  'Temp Flag',
       'Dew Point Temp (°C)',
      #  'Dew Point Temp Flag',
       'Rel Hum (%)',
      #  'Rel Hum Flag',
       'Precip. Amount (mm)',
      #  'Precip. Amount Flag',
      #  'Wind Dir (10s deg)',
      #  'Wind Dir Flag',
      #  'Wind Spd (km/h)',
      #  'Wind Spd Flag',
      #  'Visibility (km)',
      #  'Visibility Flag',
       'Stn Press (kPa)',
      #  'Stn Press Flag',
       'Hmdx',
      #  'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather'
      ]
  if split_datetime:
    df['Y'] = df['DateTime'].dt.year
    df['M'] = df['DateTime'].dt.month
    df['D'] = df['DateTime'].dt.day
    df['H'] = df['DateTime'].dt.hour
    cols = ['Y', 'M', 'D', 'H']
  else:
    cols = ['DateTime']

  # delete leap day
  df = df[~((df.DateTime.dt.month == 2) & (df.DateTime.dt.day == 29))]
  dt = df['DateTime'] # store datettime

  cols += ieso_cols+climate_cols

  df = df[cols]

  # make columns names better
  df.columns = df.columns.str.replace('.', '')
  df.columns = df.columns.str.replace(' ', '')
  df.columns = df.columns.str.replace(r"\(.*?\)", "", regex=True)

  nans = df.isna().sum().to_dict()

  # dirty mean imputation
  data = df.fillna(df.mean())

  # # dirty -1 imputation
  # data = df.fillna(pd.Series(index=df.columns, data=[-1.0]*len(df.columns)))

  data = data.reset_index()

  return df, nans, dt

df, nans, dt = preprocess(dataset)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense


data = df[['Ottawa']]


scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

train_size = int(len(data_scaled) * 0.80)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]


def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 3
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')


model.fit(X_train, Y_train, epochs=50, batch_size=200, verbose=2)


train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


train_predict = scaler.inverse_transform(np.concatenate((train_predict, np.zeros((len(train_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_train = scaler.inverse_transform(np.concatenate((Y_train.reshape(-1, 1), np.zeros((len(Y_train), len(data.columns) - 1))), axis=1))[:, 0]
test_predict = scaler.inverse_transform(np.concatenate((test_predict, np.zeros((len(test_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_test = scaler.inverse_transform(np.concatenate((Y_test.reshape(-1, 1), np.zeros((len(Y_test), len(data.columns) - 1))), axis=1))[:, 0]

#trainScore = np.sqrt(mean_squared_error(Y_train, train_predict))
#print('Train Score: %.2f RMSE' % (trainScore))
#testScore = np.sqrt(mean_squared_error(Y_test, test_predict))
#print('Test Score: %.2f RMSE' % (testScore))


In [ ]:
#@markdown plot prediction
fig = go.Figure()
fig.add_trace(go.Scattergl(
    x=dt[train_size + look_back + 1:][:len(Y_test)],
    y=Y_test,
    name='Actual',
    line_color='blue')
)

fig.add_trace(go.Scattergl(
    x=dt[train_size + look_back + 1:][:len(Y_test)],
    y=test_predict,
    name='Predicted',
    line_color='red')
)

# Set the theme to 'plotly_white'
fig.update_layout(
    title=f"Time Series Forecasting for with LSTM",
    xaxis_title="Date", # Change the x-axis title
    yaxis_title="Energy Demand",
    template="plotly_white",
    xaxis = dict(
      rangeslider=dict(
          visible=True
      ),
      tickformat="%Y-%m-%d"
    )
)
fig.show()


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error


mse = mean_squared_error(Y_test, test_predict)
print(f'Mean Squared Error (MSE): {mse}')


mae = mean_absolute_error(Y_test, test_predict)
print(f'Mean Absolute Error (MAE): {mae}')


mape = np.mean(np.abs((Y_test - test_predict) / Y_test)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')


Mean Squared Error (MSE): 922.6206744705328
Mean Absolute Error (MAE): 23.407713742709788
Mean Absolute Percentage Error (MAPE): 2.35%


In [ ]:
#@markdown merge and preprocess

dataset = pd.merge(ieso_dataset, weather_dataset, on="DateTime")

def preprocess(dataset:pd.DataFrame, split_datetime=True) -> pd.DataFrame:
  df = dataset.copy()
  ieso_cols = ['Niagara']
  climate_cols = [
       'Temp (°C)',
      #  'Temp Flag',
       'Dew Point Temp (°C)',
      #  'Dew Point Temp Flag',
       'Rel Hum (%)',
      #  'Rel Hum Flag',
       'Precip. Amount (mm)',
      #  'Precip. Amount Flag',
      #  'Wind Dir (10s deg)',
      #  'Wind Dir Flag',
      #  'Wind Spd (km/h)',
      #  'Wind Spd Flag',
      #  'Visibility (km)',
      #  'Visibility Flag',
       'Stn Press (kPa)',
      #  'Stn Press Flag',
       'Hmdx',
      #  'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather'
      ]
  if split_datetime:
    df['Y'] = df['DateTime'].dt.year
    df['M'] = df['DateTime'].dt.month
    df['D'] = df['DateTime'].dt.day
    df['H'] = df['DateTime'].dt.hour
    cols = ['Y', 'M', 'D', 'H']
  else:
    cols = ['DateTime']

  # delete leap day
  df = df[~((df.DateTime.dt.month == 2) & (df.DateTime.dt.day == 29))]
  dt = df['DateTime'] # store datettime

  cols += ieso_cols+climate_cols

  df = df[cols]

  # make columns names better
  df.columns = df.columns.str.replace('.', '')
  df.columns = df.columns.str.replace(' ', '')
  df.columns = df.columns.str.replace(r"\(.*?\)", "", regex=True)

  nans = df.isna().sum().to_dict()

  # dirty mean imputation
  data = df.fillna(df.mean())

  # # dirty -1 imputation
  # data = df.fillna(pd.Series(index=df.columns, data=[-1.0]*len(df.columns)))

  data = data.reset_index()

  return df, nans, dt

df, nans, dt = preprocess(dataset)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense


data = df[['Niagara']]


scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

train_size = int(len(data_scaled) * 0.80)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]


def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 3
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')


model.fit(X_train, Y_train, epochs=50, batch_size=200, verbose=0)


train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


train_predict = scaler.inverse_transform(np.concatenate((train_predict, np.zeros((len(train_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_train = scaler.inverse_transform(np.concatenate((Y_train.reshape(-1, 1), np.zeros((len(Y_train), len(data.columns) - 1))), axis=1))[:, 0]
test_predict = scaler.inverse_transform(np.concatenate((test_predict, np.zeros((len(test_predict), len(data.columns) - 1))), axis=1))[:, 0]
Y_test = scaler.inverse_transform(np.concatenate((Y_test.reshape(-1, 1), np.zeros((len(Y_test), len(data.columns) - 1))), axis=1))[:, 0]

#trainScore = np.sqrt(mean_squared_error(Y_train, train_predict))
#print('Train Score: %.2f RMSE' % (trainScore))
#testScore = np.sqrt(mean_squared_error(Y_test, test_predict))
#print('Test Score: %.2f RMSE' % (testScore))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2409/2409 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
603/603 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [ ]:
#@markdown plot prediction
fig = go.Figure()
fig.add_trace(go.Scattergl(
    x=dt[train_size + look_back + 1:][:len(Y_test)],
    y=Y_test,
    name='Actual',
    line_color='blue')
)

fig.add_trace(go.Scattergl(
    x=dt[train_size + look_back + 1:][:len(Y_test)],
    y=test_predict,
    name='Predicted',
    line_color='red')
)

# Set the theme to 'plotly_white'
fig.update_layout(
    title=f"Time Series Forecasting for with LSTM",
    xaxis_title="Date", # Change the x-axis title
    yaxis_title="Energy Demand",
    template="plotly_white",
    xaxis = dict(
      rangeslider=dict(
          visible=True
      ),
      tickformat="%Y-%m-%d"
    )
)
fig.show()


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error


mse = mean_squared_error(Y_test, test_predict)
print(f'Mean Squared Error (MSE): {mse}')


mae = mean_absolute_error(Y_test, test_predict)
print(f'Mean Absolute Error (MAE): {mae}')


mape = np.mean(np.abs((Y_test - test_predict) / Y_test)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')


Mean Squared Error (MSE): 1772.5029711677046
Mean Absolute Error (MAE): 33.20448298284268
Mean Absolute Percentage Error (MAPE): 7.30%
